<a href="https://colab.research.google.com/github/Greqorian/IKEA_scrapper_colab/blob/main/Copy_of_IKEA_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# package imports
#basics
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json # json files

#visualisation
from matplotlib import pyplot as plt

#scrapping
from bs4 import BeautifulSoup
import requests


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# Opening JSON file
f = open('/content/drive/MyDrive/DATA/ikea_sample_file.json', "r")
# a dictionary
data = json.load(f)

print(data[0])
print(len(data))

{'product_title': 'JÄRVFJÄLLET Office chair with armrests', 'product_url': 'https://www.ikea.com/us/en/p/jaervfjaellet-office-chair-with-armrests-60510640/', 'sku': '605.106.40', 'mpn': '605.106.40', 'currency': 'USD', 'product_price': '279', 'product_condition': 'NewCondition', 'availability': 'InStock', 'seller': 'IKEA', 'seller_url': '', 'brand': 'IKEA', 'raw_product_details': '<span class="range-revamp-product-details__paragraph">10-year Warranty. Read about the terms in the warranty brochure.</span>/<span class="range-revamp-product-details__paragraph">The adjustable and lockable tilt function increases stability and control in different sitting positions.</span>/<span class="range-revamp-product-details__paragraph">The mesh material of the backrest lets air through to your back during long sitting periods.</span>/<span class="range-revamp-product-details__paragraph">Your back gets the support it needs because the seat depth can be adjusted and the chair has built-in lumbar suppor

Writing the new array of IKEA products

In [10]:
ikeaProducts = []

for x in data:
  id = x['sku'].replace('.','')
  name = x['product_title'].split(" ")[0]
  title = x['product_title']
  url = x['product_url']
  currency = x['currency']
  price = x['product_price']

  breadcrumbs = x['breadcrumbs'].split("/")
  print(breadcrumbs)

  if len(breadcrumbs)>3:
    category = breadcrumbs[1]
    group = breadcrumbs[2]
    subgroup = breadcrumbs[3]

    ikeaProducts.append({
        'id': id, 
        'name': name,
        'title': title, 
        'url': url, 
        'currency': currency, 
        'price': price, 
        'category': category, 
        'group': group, 
        'subgroup': subgroup})

Streaming output truncated to the last 5000 lines.
['Products', 'Baby & kids', 'Baby', 'Baby textiles', 'Baby quilts & pillows', 'LENAST Crib comforter']
['Products', 'Furniture', 'Armchairs & accent chairs', 'Fabric armchairs', 'UPPLAND Armchair']
['Products', 'Kitchen & appliances', 'Kitchen systems', 'SEKTION kitchen', 'SEKTION kitchen cabinets', 'High cabinets', 'SEKTION ', ' MAXIMERA High cb w 2 doors', 'shelves', '4 drawers']
['Products', 'Storage & organization', 'TV & media furniture', 'TV & media storage', 'BRIMNES TV storage combination']
['Products', 'Kitchen & appliances', 'Kitchen systems', 'SEKTION kitchen', 'SEKTION kitchen cabinets', 'Base cabinets', 'SEKTION Base cabinet with 4 drawers']
['Products', 'Storage & organization', 'Shelving units, bookcases & storage options', 'Cube storage', 'EKET Cabinet']
['Products', 'Home Textiles', 'Window treatments', 'Curtains & drapes', 'Light filtering curtains', 'CITRUSTRÄD Curtains, 1 pair']
['Products', 'Kitchen & appliances', 

In [11]:
os.chdir('/content/drive/MyDrive/DATA')

# Check current working directory.
curDir = os.getcwd() 
print("Current working directory:", curDir)
try:
  os.mkdir(os.path.join(curDir, 'IKEAImages'))
except:
     pass

os.chdir(os.path.join(curDir, 'IKEAImages'))
print("Created new working directory:") 
!pwd

Current working directory: /content/drive/MyDrive/DATA
Created new working directory:
/content/drive/MyDrive/DATA/IKEAImages


Save the array of IKEA Products to JSON file

In [12]:
print(len(ikeaProducts))
print(ikeaProducts[0])

with open('ikeaProducts.json', 'w', encoding='utf-8') as outfile:
    json.dump(ikeaProducts, outfile, ensure_ascii=False)

11279
{'id': '60510640', 'name': 'JÄRVFJÄLLET', 'title': 'JÄRVFJÄLLET Office chair with armrests', 'url': 'https://www.ikea.com/us/en/p/jaervfjaellet-office-chair-with-armrests-60510640/', 'currency': 'USD', 'price': '279', 'category': 'Furniture', 'group': 'Chairs', 'subgroup': 'Desk chairs'}


In [13]:
# Opening JSON file
f = open('/content/drive/MyDrive/DATA/IKEAImages/ikeaProducts.json', "r")
# a dictionary
ikeaProducts = json.load(f)

In [14]:
ikeaImagesList = []
range = [0,5000]

for x in ikeaProducts[range[0]:range[1]]:
  print(x['url'])

  url = x['url']
  id = x['id']
  name = x['name']
  title = x['title']
  category = x['category']
  group = x['group']
  subgroup = x['subgroup']

  r = requests.get(url)
  soup = BeautifulSoup(r.text, 'html.parser')
  images = soup.find_all('img')

  for image in images:

    if hasattr(image, 'alt') and hasattr(image, 'src') and name in image['alt']:

      ikeaImagesList.append({
          'id': id, 
          'name': name, 
          'title': title, 
          'category':category, 
          'group':group, 
          'subgroup':subgroup, 
          'src': image['src']  
          })
      
  print(len(ikeaImagesList))    

Streaming output truncated to the last 5000 lines.
https://www.ikea.com/us/en/p/pynten-seat-pad-blue-80479238/
12964
https://www.ikea.com/us/en/p/sektion-corner-base-cab-w-pull-out-fitting-white-bodbyn-gray-s79345240/
12967
https://www.ikea.com/us/en/p/pax-3-wardrobe-frames-white-s19895339/
12971
https://www.ikea.com/us/en/p/sinarp-door-brown-20404694/
12976
https://www.ikea.com/us/en/p/sektion-maximera-high-cab-w-door-3-fronts-5-drawers-white-axstad-matte-blue-s09402601/
12980
https://www.ikea.com/us/en/p/bjoerksta-picture-and-frame-lunar-phases-black-s39416833/
12985
https://www.ikea.com/us/en/p/sektion-high-cabinet-with-shelves-2-doors-white-havstorp-beige-s29421062/
12987
https://www.ikea.com/us/en/p/sektion-maximera-high-cab-w-door-3-fronts-5-drawers-white-voxtorp-dark-gray-s09378408/
12993
https://www.ikea.com/us/en/p/sportslig-duvet-cover-and-pillowcase-running-track-40491379/
13000
https://www.ikea.com/us/en/p/gamlared-stefan-table-and-2-chairs-light-antique-stain-brown-black-s

In [15]:
print('Control positions in arrays:')
print(len(ikeaImagesList))    
print('flitered:',ikeaImagesList[20])
print('flitered:',ikeaImagesList[21])
print('flitered:',ikeaImagesList[22])
print('flitered:',ikeaImagesList[23])
print('flitered:',ikeaImagesList[24])
print('flitered:',ikeaImagesList[25])
print('flitered:',ikeaImagesList[26])
print('flitered:',ikeaImagesList[27])
print('flitered:',ikeaImagesList[28])
print('flitered:',ikeaImagesList[29])


Control positions in arrays:
25920
flitered: {'id': '90467065', 'name': 'DRÖNA', 'title': 'DRÖNA Box - black 9 ¾x13 ¾x9 ¾ "', 'category': 'Home Décor', 'group': 'Storage boxes & baskets', 'subgroup': 'Paper & media boxes', 'src': 'https://www.ikea.com/us/en/images/products/droena-box-black-white__0913367_pe785267_s5.jpg?f=xu'}
flitered: {'id': '90467065', 'name': 'DRÖNA', 'title': 'DRÖNA Box - black 9 ¾x13 ¾x9 ¾ "', 'category': 'Home Décor', 'group': 'Storage boxes & baskets', 'subgroup': 'Paper & media boxes', 'src': 'https://www.ikea.com/us/en/images/products/droena-box-dark-blue__0513338_pe638902_s5.jpg?f=xu'}
flitered: {'id': '90467065', 'name': 'DRÖNA', 'title': 'DRÖNA Box - black 9 ¾x13 ¾x9 ¾ "', 'category': 'Home Décor', 'group': 'Storage boxes & baskets', 'subgroup': 'Paper & media boxes', 'src': 'https://www.ikea.com/us/en/images/products/droena-box-patterned-light-green-white__1028977_pe835618_s5.jpg?f=xu'}
flitered: {'id': '90467065', 'name': 'DRÖNA', 'title': 'DRÖNA Box - b

In [16]:
with open('ikeaImagesList1.json', 'w', encoding='utf-8') as outfile:
    json.dump(ikeaProducts, outfile, ensure_ascii=False)

In [ ]:
for index, image in enumerate(filteredImages):
  alt = image['alt']
  link = image['src']

  fileName = str(index) + '_' + imagesIDs[index].replace('.', '') + '_' + alt.replace(' ', '_').replace('/', '-').replace(',', '').replace('"', '') +'.jpg'
  with open(fileName, 'wb') as f:
    im = requests.get(link)
    f.write(im.content)
    f.close()
    print('Writing: ', fileName)